In [9]:
from dataset import RegDataSetMindBoggle
import os
import time
import datetime
import torch
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data import DataLoader
from torchvision import transforms
import SimpleITK as sitk
import numpy as np
from Utils import *
from Loss import *

In [10]:
class SitkToTensor(object):
    """Convert sitk image to 4D Tensors with shape(1, D, H, W)"""

    def __call__(self, sample):
        self.img = sample['image']

        img_np = sitk.GetArrayFromImage(self.img)
        # threshold image intensity to 0~1
        img_np[np.where(img_np > 1.0)] = 1.0
        img_np[np.where(img_np < 0.0)] = 0.0

        img_np = np.float32(img_np)
        # img_np = np.expand_dims(img_np, axis=0)  # expand the channel dimension
        sample['image'] = torch.from_numpy(img_np).unsqueeze(0)

        if 'segmentation' in sample.keys():
            self.seg = sample['segmentation']
            seg_np = sitk.GetArrayFromImage(self.seg)
            seg_np = np.uint8(seg_np)
            sample['segmentation'] = torch.from_numpy(seg_np)

        return sample

In [11]:
train_transforms = []
train_transforms.append(SitkToTensor())
train_transforms = transforms.Compose(train_transforms)
training_data = RegDataSetMindBoggle('/home/user/Documents/dataset/Mindboggle101/mindboggle/test.txt', 
                                '/home/user/Documents/dataset/Mindboggle101/mindboggle', with_seg=True,
                                preload=False, pre_transform=train_transforms,
                                n_samples=21 * 2)
training_data_loader = DataLoader(training_data, batch_size=1,
                                               shuffle=True, num_workers=0)
train_data_iter = iter(training_data_loader)
images, truths = next(train_data_iter)

In [12]:
a1=truths[1]
print(a1)
# print(truths.shape)


tensor([[[[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         ...,

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          ...,
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 0,  ..., 0, 0, 0]],

         [[0, 0, 0,  ..., 0, 0, 0],
          [0, 0, 

In [4]:
import torch
from torch.nn import functional as F
import numpy as np
X, Y, Z = 10, 10, 10
img = np.zeros(shape=[X, Y, Z], dtype=np.float32)
for i in range(X):
    for j in range(Y):
        for k in range(Z):
            img[i,j,k] = i*100+j*10+k
inp = torch.from_numpy(img).unsqueeze(0).unsqueeze(0)
grid = torch.from_numpy(np.array([[9, 2, 3]], dtype=np.float32)).unsqueeze(1).unsqueeze(1).unsqueeze(1)
grid[..., 0] /= (X-1)
grid[..., 1] /= (Y-1)
grid[..., 2] /= (Z-1)

grid = 2*grid - 1

newgrid = grid.clone()
newgrid[..., 0] = grid[..., 2]
newgrid[..., 1] = grid[..., 1]
newgrid[..., 2] = grid[..., 0]

outp = F.grid_sample(inp, grid=newgrid, mode='bilinear', align_corners=True)
print(outp)

tensor([[[[[329.]]]]])


In [5]:
img

array([[[  0.,   1.,   2.,   3.,   4.,   5.,   6.,   7.,   8.,   9.],
        [ 10.,  11.,  12.,  13.,  14.,  15.,  16.,  17.,  18.,  19.],
        [ 20.,  21.,  22.,  23.,  24.,  25.,  26.,  27.,  28.,  29.],
        [ 30.,  31.,  32.,  33.,  34.,  35.,  36.,  37.,  38.,  39.],
        [ 40.,  41.,  42.,  43.,  44.,  45.,  46.,  47.,  48.,  49.],
        [ 50.,  51.,  52.,  53.,  54.,  55.,  56.,  57.,  58.,  59.],
        [ 60.,  61.,  62.,  63.,  64.,  65.,  66.,  67.,  68.,  69.],
        [ 70.,  71.,  72.,  73.,  74.,  75.,  76.,  77.,  78.,  79.],
        [ 80.,  81.,  82.,  83.,  84.,  85.,  86.,  87.,  88.,  89.],
        [ 90.,  91.,  92.,  93.,  94.,  95.,  96.,  97.,  98.,  99.]],

       [[100., 101., 102., 103., 104., 105., 106., 107., 108., 109.],
        [110., 111., 112., 113., 114., 115., 116., 117., 118., 119.],
        [120., 121., 122., 123., 124., 125., 126., 127., 128., 129.],
        [130., 131., 132., 133., 134., 135., 136., 137., 138., 139.],
        [140., 141

In [44]:
grid_f = generate_grid3D_tensor([10,20,30], factor=1, sq=False).unsqueeze(0)  # [-1,1]
print(grid_f.shape)
print(grid_f[0,0,0,0,:])
a1= grid_f[0,9,2,3,:]
# a1=torch.tensor([1,1,1],dtype=torch.float)
print(a1)
a1= a1.unsqueeze(0).unsqueeze(0).unsqueeze(0).unsqueeze(0)
print(a1.shape)
outp = F.grid_sample(inp, grid=grid_f, mode='bilinear', align_corners=True)
print(outp[0,0,9,2,3])
print(inp[0,0,9,2,3])
# print(inp)
print((outp-inp).max())

torch.Size([1, 10, 20, 30, 3])
tensor([-1., -1., -1.])
tensor([-0.7931, -0.7895,  1.0000])
torch.Size([1, 1, 1, 1, 3])
tensor(6.6044)
tensor(14.)
tensor(1.3018)


/home/user/anaconda3/envs/snes/lib/python3.9/site-packages/torch/nn/functional.py:4193: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
